<a href="https://colab.research.google.com/github/rmngllnn/Stage2021/blob/main/POS_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
google_colab = False

In [2]:
import pandas as pd
import spacy
import csv
import string as str
import json

from ipynb.fs.defs.fonctions_preprocess import * # à installer

#TODO : importer les fonctions de fonctions_preprocess

## Aperçu des données

28 834 entrées et 46 variables par entrée

In [10]:
df = pd.read_csv("../DEMOCRATIE_ET_CITOYENNETE.csv", low_memory = False)
df.shape

#df.head()

(116549, 48)

## Extraction et Nettoyage des données

https://docs.python.org/fr/3/library/csv.htmlhttps://docs.python.org/fr/3/library/csv.html

### Extraction des données

In [3]:
#Extraction des données dans une liste de listes
#len(ex) = 28385 soit le nombre d’observation

csv.field_size_limit(100000000) # Adapte la taille limite autorisée

data = open_doc_csv ("../DEMOCRATIE_ET_CITOYENNETE.csv")

In [4]:
print(len(data))

116550


In [5]:
for obs in data[0]:
    print(obs)

id
reference
title
createdAt
publishedAt
updatedAt
trashed
trashedStatus
authorId
authorType
authorZipCode
QUXVlc3Rpb246MTA3 - En qui faites-vous le plus confiance pour vous faire représenter dans la société et pourquoi ?
QUXVlc3Rpb246MTA4 - En dehors des élus politiques, faut-il donner un rôle plus important aux associations et aux organisations syndicales et professionnelles ?
QUXVlc3Rpb246MTA5 - Si oui, à quel type d'associations ou d'organisations ? Et avec quel rôle ?
QUXVlc3Rpb246MTEw - Que faudrait-il faire pour renouer le lien entre les citoyens et les élus qui les représentent ?
QUXVlc3Rpb246MTEx - Le non-cumul des mandats instauré en 2017 pour les parlementaires (députés et sénateurs) est :
QUXVlc3Rpb246MTEy - Pourquoi ?
QUXVlc3Rpb246MTEz - Que faudrait-il faire pour mieux représenter les différentes sensibilités politiques ?
QUXVlc3Rpb246MTE0 - Pensez-vous qu'il serait souhaitable de réduire le nombre d'élus (hors députés et sénateurs) ?
QUXVlc3Rpb246MTE1 - Si oui, lesquels 

### Nettoyage des données

In [6]:
cleaned_data = clean_data(data)

## POS-Tagging using SpaCy

In [ ]:
#NE PAS RUN cette cellule sauf sur Google Colab
#Permet d'installer le module sur le serveur distant utilisé.

if google_colab:
  try:
    nlp = spacy.load("fr_core_news_sm", disable=['ner'])
  except:
    from spacy.cli import download
    download('fr_core_news_sm')
    nlp = spacy.load("fr_core_news_sm", disable=['ner'])

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [48]:
nlp = spacy.load("fr_core_news_sm")
#doc = nlp(d) # /!\ Doit être de type String

def pos_tagger(data_list):
    
    """ Tag each token of data in input
    -> list of sentences
    <- list of nouns, adjectives and pronouns with lemma of each token
    """
  
    nouns = []
    adj = []
    pronouns = []
    
    loop = 100000
    
    def tag(data_item):
        doc = nlp(data_item)
        for token in doc:
            if token.pos_ == "NOUN" : nouns.append((token.text, token.lemma_))
            elif token.pos_ == "ADJ" : adj.append((token.text, token.lemma_))
            elif token.pos_ == "PRON" : pronouns.append((token.text, token.lemma_))
                
    for i in range (0, len(data_list)-loop, loop):
        print("Nous en sommes à" , i, "/", len(data_list))
        for j in range (i, i+loop):
            tag(data_list[j])
    print("Nous en sommes à" , loop*(len(data_list)//loop), "/", len(data_list))
    for j in range(loop*(len(data_list)//loop), len(data_list)):
        tag(data_list[j])
    #if i%100000 == 0 : print(i, "/", len(data_list))
    
    return nouns, adj, pronouns

In [47]:
pos_tagger(cleaned_data)

NameError: name 'message' is not defined

## Sérialisation des données

Données sérialisées au format json car facilement lisible hors programmation  

https://www.codeflow.site/fr/article/python-json  

La cellule ci-dssous ne doit être runé qu'une seule fois

In [ ]:
def serialisation_data (data, title):
  """
  Serialize data in a json file
  -> Title mus be a string : title.json
  <- Save a file in desktop
  """

  with open(title, "w+") as file:
    json.dump(data, file)

'with open("nouns.json", "w+") as fichier:\n  json.dump(nouns, fichier)\n\nwith open("adjectives.json", "w+") as fichier:\n  json.dump(adj, fichier)\n \nwith open("pronouns.json", "w+") as fichier:\n  json.dump(pronouns, fichier)\n  #fichier.write(pronouns)'

In [ ]:
serialisation_data(a, "test.json")